<a href="https://colab.research.google.com/github/duckyngo/Word-Error-Rate-Visualization-with-Colab/blob/main/1.WER_Calculation_And_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FROM: https://github.com/duckyngo/Word-Error-Rate-Visualization-with-Colab

In [7]:
import json
from IPython.display import HTML, display

try:
    import jiwer
except:
    !pip install jiwer
    import jiwer


def set_css():
    display(
        HTML("""
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """)
    )


# Turn on line wrapping on Colab Ref: https://github.com/jupyter/notebook/issues/6274
get_ipython().events.register("pre_run_cell", set_css)


def wer(ref, hyp, debug=False):
    r = ref.split()
    h = hyp.split()
    # costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h) + 1)] for outer in range(len(r) + 1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h) + 1)] for outer in range(len(r) + 1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY = 1  # Tact
    INS_PENALTY = 1  # Tact
    SUB_PENALTY = 1  # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r) + 1):
        costs[i][0] = DEL_PENALTY * i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                costs[i][j] = costs[i - 1][j - 1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = (
                    costs[i - 1][j - 1] + SUB_PENALTY
                )  # penalty is always 1
                insertionCost = costs[i][j - 1] + INS_PENALTY  # penalty is always 1
                deletionCost = costs[i - 1][j] + DEL_PENALTY  # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        lines = []
        compares = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i -= 1
            j -= 1
            if debug:
                lines.append("OK\t" + r[i] + "\t" + h[j])
                compares.append(colored(0, 0, 0, h[j]))
        elif backtrace[i][j] == OP_SUB:
            numSub += 1
            i -= 1
            j -= 1
            if debug:
                lines.append("SUB\t" + r[i] + "\t" + h[j])
                compares.append(
                    colored(0, 255, 0, h[j]) + colored(0, 0, 0, f"({r[i]})")
                )
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j -= 1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
                compares.append(colored(0, 0, 255, h[j]))
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i -= 1
            if debug:
                lines.append("DEL\t" + r[i] + "\t" + "****")
                compares.append(colored(255, 0, 0, r[i]))
    if debug:
        # print("OP\tREF\tHYP")
        # lines = reversed(lines)
        # for line in lines:
        #     print(line)

        compares = reversed(compares)
        for line in compares:
            print(line, end=" ")
        # print("Ncor " + str(numCor))
        # print("Nsub " + str(numSub))
        # print("Ndel " + str(numDel))
        # print("Nins " + str(numIns))
    wer_result = round((numSub + numDel + numIns) / (float)(len(r)), 3)
    return {
        "WER": wer_result,
        "Cor": numCor,
        "Sub": numSub,
        "Ins": numIns,
        "Del": numDel,
    }, compares


def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)


def strike(text, color=None):
    if color:
        return colored(0, 255, 0, "".join(["\u0336{}".format(c) for c in text]))

    else:
        return colored(0, 0, 0, "".join(["\u0332{}".format(c) for c in text]))


# @title Calculate and visualize WER { run: "auto" }

remove_punctuation = True  # @param {type:"boolean"}
input_ref = "Dank u wel. De heer Visser, Christen Unie. Dank u wel, voorzitter. Wat betreft de oude stukken die opnieuw geheim verklaaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. Zo snel mogelijk in het najaar gaan we er opnieuw over hebben. Wat betreft het PWC-rapport is niet vanaf vandaag het openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel dat we vanavond toch al op het zuid gaan komen. Dus waarom zouden we dat nu nog geheim verklaaren? Ik ben er niet van overtuigd wat daar het argument voor is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. Het is tot en met een ding niet akkoord. Al was het maar voor de symboolwerking. Maar bijlagen zeven bij de gedemd oudergeracht de uitgiftetekening. Ik heb net al gezegd ik vind het compleet een nonsens om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de Raad. Ja, voorzitter. Wij gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd. Dat hebben we bij het seniorenconvental aangegeven dat we daar niet meer van gediend zijn. Voor de rest heeft u gevraagd om alles in een keer te behandelen. U heeft ook gezegd alles komt terug. Dus u behandelt alles in een keer en het feit dat het terugkomt vinden wij dat niks geheim verklaard hoeft te worden. Dank u.".lower()  # @param {type:"string"}
input_hyp = "Dank u wel. De heer visser, christen unie. Dank u wel voorzitter, wat betreft de oude stukken die opnieuw geheim verklaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. zo snel mogelijk in het najaar gaan we het er opnieuw over hebben. wat betreft het pwcrapport is in ieder geval vanaf vandaag openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft Dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel er wordt vanavond gewoon een besluit genomen, dus waarom zouden we dat nu nog geheim verklaren? Ik ben er niet van overtuigd wat daar het argument van is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. En tenslotte ga ik met een ding niet akkoord, al was het maar voor de symboolwerking. Maar bijlage zeven bij de gedemde ouderdrachte uitgiftetekening. Ik heb het net al gezegd, ik vind het complete onzin om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de raad. Ja voorzitter we gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd dat hebben we bij het seniorenconferent al aangegeven dat we daar niet meer van gediend zijn. En voor de rest heeft u gevraaagd om alles in een keer te behandelen en u heeft ook gezegd alles komt terug, dus ik behandel alles in een keer, en het feit dat het terug komt vinden wij dat niks geheim verklaard hoeft te worden dank u.".lower()  # @param {type:"string"}
input_json = ""  # @param {type:"string"}

if input_json and input_json != "":
    json_data = json.loads(input_json)
    input_ref = json_data["text"]
    input_hyp = json_data["pred_text"]


if remove_punctuation == True:
    ref = jiwer.RemovePunctuation()(input_ref)
    hyp = jiwer.RemovePunctuation()(input_hyp)
else:
    ref = input_ref
    hyp = input_hyp

print(f"REF: {ref}\n")
print(f"HYP: {hyp}")
print("-" * 30)

output, compares = wer(ref, hyp, debug=True)

print()
print(colored(0, 0, 0, f"N CORRECT   : {output['Cor']}"))
print(colored(255, 0, 0, f"N DELETE    : {output['Del']}"))
print(colored(0, 255, 0, f"N SUBSTITUTE: {output['Sub']}"))
print(colored(0, 0, 255, f"N INSERT    : {output['Ins']}"))
print(colored(0, 0, 0, f"WER: {output['WER']}"))

REF: dank u wel de heer visser christen unie dank u wel voorzitter wat betreft de oude stukken die opnieuw geheim verklaaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op zo snel mogelijk in het najaar gaan we er opnieuw over hebben wat betreft het pwcrapport is niet vanaf vandaag het openbaar met uitzondering van de weggelakte delen dus dat is ook mooi wat betreft dateterrein ben ik toch een beetje benieuwd naar de reactie van het college en ik heb inderdaad ook het gevoel dat we vanavond toch al op het zuid gaan komen dus waarom zouden we dat nu nog geheim verklaaren ik ben er niet van overtuigd wat daar het argument voor is dus ik zou daar eigenlijk een kort antwoord van het college op willen het is tot en met een ding niet akkoord al was het maar voor de symboolwerking maar bijlagen zeven bij de gedemd oudergeracht de uitgiftetekening ik heb net al gezegd ik vind het compleet een nonsens om zon tekening geheim te verklaren het college mag me wel eens u